In [1]:
import torch

In [2]:
#devito set up
from abc import ABC, abstractmethod
from devito import Operator, Function
from numpy import array
import numpy as np
from devito import Grid, Function, dimensions, Eq, Inc
import sympy

In [62]:
batch_weights = torch.rand(2,3,3,3)

# python implementation

In [489]:
def batch_norm(weight):
    eps = 1e-5
    momentum = 0.1 #i dont know where it goes
    Var = torch.var(weight, unbiased=False, axis = (0,2,3))
    E = torch.mean(weight, axis = (0,2,3))
    print("E:", E, "Var",Var)
    gamma = 1
    betta = 0
    result = torch.zeros_like(batch_weights)
    for image in range(weight.shape[0]):
        for channel in range(weight.shape[1]):
            for x in range(weight.shape[2]):
                for y in range(weight.shape[3]):
                    result[image][channel][x][y]=(batch_weights[image][channel][x][y]-mean3chan[channel])\
                    /(np.sqrt(var3chan[channel]+1e-5))
    return result
res = batch_norm(batch_weights)

E: tensor([0.6089, 0.5388, 0.5146]) Var tensor([0.0834, 0.0535, 0.0765])


# pytorch implementation

In [490]:
m = torch.nn.BatchNorm2d(3, affine=False)
output = m(batch_weights)

In [491]:
#compare
np.allclose(res, output, atol = 1)

False

# Devito implementation

Loops to find the variance and the mean. Consider implementing them in devito

In [390]:
#find the means
images, channels, height, width = batch_weights.shape
means = np.zeros(channels)
#838 µs for 7 runs, 1000 loops each
for channel in range(channels):
    for image in range(images):
        means[channel] += sum(sum(batch_weights[image][channel]))/(height*width)
means/=images
means

array([0.60885847, 0.53883916, 0.51463312])

In [408]:
#find std
var = np.zeros(channels)
for channel in range(channels):
    for image in range(images):
            var[channel] += sum(sum(pow(batch_weights[image][channel]-means[channel],2)))/(height*width)
var/=images
var

array([0.08341521, 0.05345532, 0.07645875])

In [494]:
#devito
a, b, c, d = dimensions('a b c d')
gridB = Grid(shape=(2,3,3,3), dimensions = (a, b, c, d) )
B = Function(name='B', grid=gridB, space_order=0)

B.data[:] = batch_weights[:]
e, f, g, h = dimensions('e f g h')
gridR = Grid(shape=(2,3,3,3), dimensions = (e, f, g, h) )
R = Function(name='R', grid=gridR, space_order=0)

R.data[:] = 0

images, channels, height, width = batch_weights.shape
gridMean = Grid(shape=(channels))
M = Function(name= 'M', grid=gridMean,space_order=0)
#find the means

means = np.zeros(channels)
#838 µs for 7 runs, 1000 loops each
for channel in range(channels):
    for image in range(images):
        means[channel] += sum(sum(batch_weights[image][channel]))/(height*width)
means/=images
means
M.data[:]=means[:]
# var
gridVar = Grid(shape=(channels))
V = Function(name= 'V', grid=gridVar,space_order=0)
#find var
"""
var = np.zeros(channels)
for channel in range(channels):
    for image in range(images):
            var[channel] += sum(sum(pow(batch_weights[image][channel]-means[channel],2)))/(height*width)
            
var/=images
"""
var = torch.var(batch_weights, axis = (0,2,3))
V.data[:] = var[:]

epsilon = 1e-5


rhs = ((B[a,b,c,d]-M[b])/(((V[b]+epsilon)**(-0.5))))

op = Operator(Eq(R[a, b, c, d], rhs))
op
print(op)

#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"
#include "omp.h"

struct dataobj
{
  void *restrict data;
  int * size;
  int * npsize;
  int * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
} ;


int Kernel(struct dataobj *restrict B_vec, struct dataobj *restrict M_vec, struct dataobj *restrict R_vec, struct dataobj *restrict V_vec, const int a_M, const int a_m, const int b_M, const int b_m, const int c_M, const int c_m, const int d_M, const int d_m, struct profiler * timers, const int nthreads_nonaffine)
{
  float (*restrict B)[B_vec->size[1]][B_vec->size[2]][B_vec->size[3]] __attribute__ ((aligned (64))) = (float (*)[B_vec->size[1]][B_vec->size[2]][B_vec->size[3]]) B_vec->data;
  float (*restrict M) __attribute__ ((aligned (64))) = (float (*)) M_vec->data;
  float (*restrict R)[R_vec->size[1]][R_vec->size[2]][R_vec->size[3]] __attribute__ ((aligned (

In [477]:
summary = op.apply()

Operator `Kernel` run in 0.01 s


In [481]:
R.data

Data([[[[ 0.00533423,  0.07643209,  0.10509071],
        [-0.00995114, -0.01896134,  0.11199094],
        [ 0.00178706,  0.10736525, -0.1651808 ]],

       [[-0.02406164,  0.00917829, -0.00512178],
        [ 0.05895625, -0.05264197,  0.03859347],
        [ 0.06743717,  0.10737912, -0.09726514]],

       [[ 0.03271969,  0.10236179,  0.00314345],
        [ 0.00800933, -0.11866892, -0.03102242],
        [-0.11613946,  0.08853134, -0.09329361]]],


      [[[-0.10346019,  0.01344351, -0.12639934],
        [-0.1381232 ,  0.02321276,  0.05379003],
        [-0.06567074,  0.03420006,  0.09510005]],

       [[ 0.02302079,  0.02187578,  0.02959034],
        [-0.03607467, -0.07867918, -0.06373384],
        [-0.05182636, -0.01176433,  0.0651378 ]],

       [[ 0.09435913,  0.11328796, -0.11915923],
        [-0.05221762,  0.01927992,  0.01662871],
        [ 0.07543337, -0.08040615,  0.05715271]]]], dtype=float32)

In [484]:
res - R.data

tensor([[[[-0.1919, -0.1434, -0.1239],
          [-0.2024, -0.2085, -0.1192],
          [-0.1944, -0.1223, -0.3083]],

         [[-0.6746, -0.6454, -0.6580],
          [-0.6016, -0.6998, -0.6195],
          [-0.5941, -0.5590, -0.7390]],

         [[-1.0004, -0.9701, -1.0133],
          [-1.0112, -1.0662, -1.0281],
          [-1.0651, -0.9762, -1.0552]]],


        [[[-0.2662, -0.1864, -0.2818],
          [-0.2898, -0.1797, -0.1589],
          [-0.2404, -0.1722, -0.1307]],

         [[-0.6332, -0.6342, -0.6274],
          [-0.6852, -0.7227, -0.7095],
          [-0.6990, -0.6638, -0.5962]],

         [[-0.9736, -0.9654, -1.0664],
          [-1.0373, -1.0063, -1.0074],
          [-0.9819, -1.0496, -0.9898]]]])